<a href="https://colab.research.google.com/github/reidnclark/MFDPRepo/blob/main/mean_reversion_trading_algorithm_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multipurpose Financial Data Processor (MFDP)**
---

### Contents Summary:

|Analysis Section|Values|
|-|-|
|**1. Company Overview**|- Competitive Position|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**2. Financial Statements Analysis**|- **Income Statement**|
||----- Revenue Growth Rate|
||----- Profit Margins (Gross, Operating, Net)|
||----- Earnings Per Share (EPS) & EPS Growth|
||- **Balance Sheet**|
||----- Debt-to-Equity Ratio|
||- **Cash Flow Statement**|
||----- Cash Flow from Operating Activities|
||----- Cash Flow from Investing and Financing Activities|
||----- Free Cash Flow Analysis|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**3. Key Financial Ratios**|- **Profitability Ratios**|
||----- Return-on-Equity (ROE)|
||----- Return-on-Assets (ROA)|
||----- Net Profit Margin|
||- **Liquidity Ratios**|
||----- Current Ratio|
||----- Quick Ratio|
||- **Efficiency Ratios**|
||----- Asset Turnover Ratio|
||----- Inventory Turnover Ratio|
||- **Leverage Ratios**|
||----- Debt-to-Equity Ratio|
||----- Interest Coverage Ratio|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**3. Valuation**|- Price-to-Earnings|
||- Price-to-Book|
||- Discounted Cash Flow (DCF) Analysis|
||- **Other Valuation Models**|
||----- Dividend Discount Model (DDM)|
||----- EBITDA Multiples|
|------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
|**4. Qualitative Analysis**|- Price-to-Earnings|
||- Price-to-Book|
||- Discounted Cash Flow (DCF) Analysis|
||- **Other Valuation Models**|
||----- Dividend Discount Model (DDM)|
||----- EBITDA Multiples|

---
# **1) Fundamental Analysis**

#### Import Libraries:

In [1106]:
import yfinance as yf
import pandas as pd
from babel.numbers import format_currency, format_percent

---
#### **Input Arguments**:

# Long Stocks:
- ### Siemens Energy AG (ENR)
- ### Nutrien (NTR)
- ### Shin-Etsu Chemical Co. Ltd. (SHECY)
- ### Taiwan Semiconductor Mfg. Co. Ltd. (TSM)
- ### Canadian Natural Resources Limited (CNQ)
- ### Dollarama (DOL.TO)


---

#### Inputs:

In [1121]:
ticker_input = 'DOL.TO'
financial_attributes_input = ['EBITDA'] # just use one for forecaster
cashflow_attributes_input = ['Free Cash Flow'] # ditto above
balsheet_attributes_input = ['Total Debt']
basic_attributes_input = ['beta', 'marketCap', 'sharesOutstanding']

---
# **2) DCF Model**

#### Get Data Function:

In [1122]:
def get_basic_data(attributes_data, attributes_input):
    basic_data_dict = {}
    for attr in attributes_input:
      basic_data_dict[attr] = attributes_data.get(attr)
    return basic_data_dict

def get_attr_hist_data(attributes_data, attributes_input):
    for attr in attributes_input:
      attr_hist_data = attributes_data.loc[attr].dropna()
    return attr_hist_data

def forecast(attr_hist_data):
    initial_value = attr_hist_data.iloc[-1]
    ending_value = attr_hist_data.iloc[0]
    #compound annual growth rate
    cagr = ((ending_value / initial_value)**(1/len(attr_hist_data))) - 1
    print(f'cagr of {attr_hist_data.name}: {cagr*100}%')
    forecasted_list = []
    for i in range(len(attr_hist_data)):
      forecast_value = ending_value * ((1+cagr)**(i+1))
      forecasted_list.append(forecast_value)
    return forecasted_list

def get_data(ticker_input: tuple,
             financial_attributes_input: list[str],
             cashflow_attributes_input: list[str],
             balsheet_attributes_input: list[str],
             basic_attributes_input: list[str]):

  yf_ticker = yf.Ticker(ticker_input)

  financial_attributes = yf_ticker.financials
  financial_attributes_hist_data = get_attr_hist_data(financial_attributes, financial_attributes_input)

  forecasted_list_financials = forecast(financial_attributes_hist_data)

  cashflow_attributes = yf_ticker.cashflow
  cashflow_attributes_hist_data = get_attr_hist_data(cashflow_attributes, cashflow_attributes_input)
  forecasted_list_cashflow = forecast(cashflow_attributes_hist_data)

  balsheet_attributes = yf_ticker.balance_sheet
  balsheet_attributes_hist_data = get_attr_hist_data(balsheet_attributes, balsheet_attributes_input).iloc[0]

  basic_attributes = yf_ticker.info
  basic_attributes_hist_data = get_basic_data(basic_attributes, basic_attributes_input)

  return forecasted_list_financials, forecasted_list_cashflow, balsheet_attributes_hist_data, basic_attributes_hist_data, yf_ticker

forecasted_list_financials, forecasted_list_cashflow, balsheet_attributes_hist_data, basic_attributes_hist_data, yf_ticker = get_data(ticker_input,
                                                                           financial_attributes_input,
                                                                           cashflow_attributes_input,
                                                                           balsheet_attributes_input,
                                                                           basic_attributes_input
                                                                                      )


interest_expense = (yf_ticker.income_stmt.loc['Interest Expense'].iloc[0])

beta = basic_attributes_hist_data['beta']
total_equity = basic_attributes_hist_data['marketCap']
total_shares = basic_attributes_hist_data['sharesOutstanding']

total_debt = (yf_ticker.balance_sheet.loc['Total Debt'].iloc[0])
cost_of_debt = interest_expense / total_debt
total_cash = (yf_ticker.balance_sheet.loc['Cash And Cash Equivalents'].iloc[0])

cagr of EBITDA: 13.676499242077899%
cagr of Free Cash Flow: 14.806406302251318%


#### Get Beta, Total Equity, Total Debt, Cost of Debt, Cash, Shares Outstanding:

##### CAPM Formula Function:
#### Compute the Expected Eeturn ( *E ( x )* )of investment given:
---

#### 1. **Risk-free interest Rate (R)** (% Yield on a 1-yr Government Bond)
#### 2. **Beta (B)**



In [1123]:
r_f = 0.0475 # risk-free rate ( return subscript f, % )
b_i = beta # beta ( beta subscript i, ration )
e__r_m = 0.10 # expected market return ( e(return subscript m, %) )

e__r_i = r_f + b_i*(e__r_m - r_f) # expected return on investment
e__r_i

0.0766375

#### **WACC** Formula Function:
*Also known as "Weighted Average Cost of Capital"*

---

In [1124]:
E = total_equity
D = total_debt
R_e = e__r_i # capm result
R_d = cost_of_debt

T = 0.25 # tax rate

wacc = (E / (E+D))*R_e + (D / (E+D))*R_d*(1-T) # weighted average cost of capital
wacc

0.07192035399638713

#### **Terminal Value (TV)** Formula Function:

---

In [1125]:
fcf_n = forecasted_list_cashflow[-1] # value of final year in fcf forecast
g = 0.03 # perpetual growth rate (can use GDP growth estimate per year, (3 %), or ind.)

tv = (fcf_n * (1+g)) / (wacc - g)

def format_dollar(amount):
    """Format an integer as a dollar amount."""
    return f"${amount:,.2f}"

format_dollar(tv)
# in later version, add in tv financial metric multiple to find average. for now can just use perp.

'$53,449,913,358.98'

#### **Discount** Function and Ent, Eq:

---

In [1126]:
pv_of_fcfs = []
discount_factors = []
for i in range(len(forecasted_list_cashflow)):
  discount_factor = (1 / ((1+wacc)**(i+1)))
  discount_factors.append(discount_factor)
  fcf_at_i = discount_factor * forecasted_list_cashflow[i]
  pv_of_fcfs.append(fcf_at_i)

pv_of_tv = discount_factors[-1]*tv

enterprise_value = (sum(pv_of_fcfs)) + pv_of_tv

equity_value = enterprise_value - total_debt + total_cash

share_price = equity_value / total_shares
#share_price
#pv_of_fcfs
#tv
#pv_of_tv

#### **Resulting Share Price**:

---

In [1127]:
print(f'Intrinsic Price: ${share_price}')
print('------------------------------------')
current_price = yf_ticker.history(period='1d')['Close'].iloc[0]
print(f'Current Price: ${round((current_price),2)}')

Intrinsic Price: $150.57746293844858
------------------------------------
Current Price: $140.13
